<a href="https://colab.research.google.com/github/Rudra-prasad-tarai/CreditRiskOptimisation/blob/main/Credit_Risk_Optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/Rudra-prasad-tarai/CreditRiskOptimisation
!pip install odfpy

Cloning into 'CreditRiskOptimisation'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 26 (delta 7), reused 9 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 39.26 KiB | 2.31 MiB/s, done.
Resolving deltas: 100% (7/7), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160673 sha256=9b99f6662123b1cc3c1ccecaacd7445791cf78bd1f16bc32c97fe0e10e2e4914
  Stored in directory: /root/.cache/pip/wheels/d6/1d/c8/8c29be1d73ca42d15977c75193d9f39a98499413c2838ac54c
Successfully built odfpy


In [4]:
import pandas as pd
import numpy  as np

In [5]:
transition_matrix = pd.read_excel('/content/CreditRiskOptimisation/data/CrisilTransitionMatrix.ods',engine= 'odf')

In [6]:
bonds = pd.read_csv('/content/CreditRiskOptimisation/data/MW-Bonds-on-CM-18-Apr-2025.csv')


In [7]:
bonds.columns

Index(['SYMBOL \n', 'SERIES \n', 'BOND TYPE \n', 'COUPON RATE \n',
       'FACE VALUE \n', 'LTP \n', '%CHNG \n', 'VOLUME \n(Shares)',
       'VALUE \n (₹ Crores)', ' \n', 'CREDIT RATING \n', 'MATURITY DATE \n'],
      dtype='object')

In [8]:
bonds.isnull().sum()

,0
SYMBOL \n,0
SERIES \n,15
BOND TYPE \n,118
COUPON RATE \n,0
FACE VALUE \n,0
LTP \n,0
%CHNG \n,0
VOLUME \n(Shares),0
VALUE \n (₹ Crores),0
\n,398


In [9]:
bonds['VOLUME \n(Shares)'].unique()

array(['5,678', '4,469', '4,008', '2,515', '2,760', '2,163', '2,000',
       '1,308', '1,039', '210', '1,181', '930', '1,007', '903', '1,001',
       '900', '174', '909', '636', '532', '534', '535', '501', '399',
       '500', '5', '82', '507', '432', '339', '440', '1,321', '401',
       '350', '355', '304', '300', '310', '250', '330', '275', '274',
       '284', '255', '256', '853', '270', '204', '180', '200', '185',
       '163', '170', '152', '150', '160', '149', '137', '140', '142',
       '128', '130', '132', '102', '112', '105', '190', '100', '99', '98',
       '92', '90', '74', '76', '70', '68', '62', '60', '50', '78', '120',
       '40', '37', '47', '30', '27', '25', '22', '19', '21', '20', '16',
       '67', '9', '15', '10', '42', '34', '4', '6', '2', '1', '-'],
      dtype=object)

In [10]:
del bonds[bonds.columns[8]]
bonds.dropna(subset = ['CREDIT RATING \n'],inplace=True) #removed those whoo does not having crisil rating
bonds['VOLUME \n(Shares)'] = bonds['VOLUME \n(Shares)'].str.replace('-', '0')

In [11]:
bonds.columns

Index(['SYMBOL \n', 'SERIES \n', 'BOND TYPE \n', 'COUPON RATE \n',
       'FACE VALUE \n', 'LTP \n', '%CHNG \n', 'VOLUME \n(Shares)', ' \n',
       'CREDIT RATING \n', 'MATURITY DATE \n'],
      dtype='object')

In [12]:
import re

# Extract ONLY CRISIL ratings (case-insensitive, handles variations)
bonds['CREDIT RATING \n'] = bonds['CREDIT RATING \n'].str.extract(r'(CRISIL\s*[A-Za-z+-]+\s*[A-Za-z]*)', flags=re.IGNORECASE)

# Set non-CRISIL entries to NaN
bonds.loc[~bonds['CREDIT RATING \n' ].str.contains('CRISIL', na=False, case=False), 'CREDIT RATING'] = pd.NA

bonds['VOLUME \n(Shares)'] = bonds['VOLUME \n(Shares)'].str.replace(',', '').astype(int)  # or `float`

In [13]:
bonds.dropna(subset = ['CREDIT RATING \n'],inplace=True)

In [14]:
# from google.colab import files
# df = bonds['CREDIT RATING \n']
# df.to_csv('dat.csv')
# files.download('dat.csv')

In [15]:
bonds['CREDIT RATING \n'].unique()

array(['CRISIL AAA STABLE', 'CRISIL AA Stable', 'CRISIL AA Negative',
       'CRISIL AA-Positive ', 'CRISIL A', 'CRISIL AAA ',
       'CRISIL A Stable', 'CRISIL AA STABLE', 'CRISIL AA+ STABLE',
       'CRISIL AAA Stable', 'CRISIL AA', 'CRISIL A-', 'CRISIL AA-',
       'CRISIL A+'], dtype=object)

In [16]:
# Filter by credit rating
# bonds= bonds[bonds['CREDIT RATING \n'].isin(['AAA', 'AA+', 'AA', 'A+', 'A','AA ' ])]

# Filter by maturity (1–5 years from now)
import datetime as dt
today = dt.datetime.today()
cutoff = today + dt.timedelta(days=5*365)
bonds['MATURITY DATE \n'] = pd.to_datetime(bonds['MATURITY DATE \n'], errors='coerce')
bonds = bonds[(bonds['MATURITY DATE \n'] >= today) & (bonds['MATURITY DATE \n'] <= cutoff)]

# Filter by volume
bonds = bonds[bonds['VOLUME \n(Shares)'] > 5]

# You can manually tag 'Issuer Type' based on Symbol or ISIN
bonds_filtered = bonds[['SYMBOL \n', 'CREDIT RATING \n', 'COUPON RATE \n', 'LTP \n', 'MATURITY DATE \n', 'VOLUME \n(Shares)']]
bonds_filtered


,SYMBOL \n,CREDIT RATING \n,COUPON RATE \n,LTP \n,MATURITY DATE \n,VOLUME \n(Shares)
2,96IIFL28,CRISIL AA Stable,9.60,952.00,2028-11-03,4008
3,734IRFC28,CRISIL AAA STABLE,7.34,"1,078.00",2028-02-19,2515
4,96IIFL28A,CRISIL AA Negative,9.60,956.00,2028-06-24,2760
5,871REC28,CRISIL AAA STABLE,8.71,"1,123.36",2028-09-24,2163
9,893NHB29,CRISIL AAA STABLE,8.93,"5,570.00",2029-03-24,210
10,1003ISFL28,CRISIL AA-Positive,10.03,990.00,2028-12-21,1181
11,875NHAI29,CRISIL AAA STABLE,8.75,"1,146.06",2029-02-05,930
14,1065NFL27,CRISIL A,10.65,"1,002.00",2027-03-13,1001
16,888NHB29,CRISIL AAA STABLE,8.88,"5,600.00",2029-01-13,174
21,863IRFC29,CRISIL AAA STABLE,8.63,"1,116.00",2029-03-26,535


In [17]:
# Example raw ratings from your dataset
raw_ratings = [                                            # observed from the filtererd data set
    'CRISIL AAA STABLE', 'CRISIL AA Stable', 'CRISIL AA Negative',
    'CRISIL AA-Positive ', 'CRISIL A', 'CRISIL AAA ',
    'CRISIL A Stable', 'CRISIL AA STABLE', 'CRISIL AA+ STABLE',
    'CRISIL AAA Stable', 'CRISIL AA', 'CRISIL A-', 'CRISIL AA-',
    'CRISIL A+'
]

# Define a mapping function
def normalize_rating(rating):
    rating = rating.strip().upper()
    if 'AAA' in rating:
        return 'AAA'
    elif 'AA' in rating:
        return 'AA'
    elif 'A' in rating:
        return 'A'
    elif 'BBB' in rating:
        return 'BBB'
    elif 'BB' in rating:
        return 'BB'
    elif 'B' in rating:
        return 'B'
    elif 'C' in rating:
        return 'C'
    elif 'D' in rating:
        return 'D'
    else:
        return 'Other'



In [18]:
bonds_filtered['MAPPED RATING'] = bonds_filtered['CREDIT RATING \n'].map(normalize_rating)

<ipython-input-18-bc743143082e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bonds_filtered['MAPPED RATING'] = bonds_filtered['CREDIT RATING \n'].map(normalize_rating)


In [22]:
bonds_filtered

,SYMBOL \n,CREDIT RATING \n,COUPON RATE \n,LTP \n,MATURITY DATE \n,VOLUME \n(Shares),MAPPED RATING
2,96IIFL28,CRISIL AA Stable,9.60,952.00,2028-11-03,4008,AA
3,734IRFC28,CRISIL AAA STABLE,7.34,"1,078.00",2028-02-19,2515,AAA
4,96IIFL28A,CRISIL AA Negative,9.60,956.00,2028-06-24,2760,AA
5,871REC28,CRISIL AAA STABLE,8.71,"1,123.36",2028-09-24,2163,AAA
9,893NHB29,CRISIL AAA STABLE,8.93,"5,570.00",2029-03-24,210,AAA
10,1003ISFL28,CRISIL AA-Positive,10.03,990.00,2028-12-21,1181,AA
11,875NHAI29,CRISIL AAA STABLE,8.75,"1,146.06",2029-02-05,930,AAA
14,1065NFL27,CRISIL A,10.65,"1,002.00",2027-03-13,1001,A
16,888NHB29,CRISIL AAA STABLE,8.88,"5,600.00",2029-01-13,174,AAA
21,863IRFC29,CRISIL AAA STABLE,8.63,"1,116.00",2029-03-26,535,AAA


In [19]:
transition_matrix.head(10)

,Rating Category,Issuer-months,AAA,AA,A,BBB,BB,B,C,D
0,AAA,15796.0,98.83,1.16,0.01,0.00,0.00,0.00,0.00,0.00
1,AA,40980.0,2.26,96.10,1.53,0.04,0.01,0.00,0.00,0.05
2,A,78111.0,0.14,3.39,93.13,3.13,0.11,0.02,0.01,0.07
3,BBB,211375.0,0.00,0.04,3.20,91.50,4.61,0.11,0.03,0.52
4,BB,308532.0,0.00,0.00,0.01,4.14,88.97,3.73,0.15,3.01
5,B,241508.0,0.00,0.00,0.00,0.03,8.83,82.49,0.40,8.52
6,C,5330.0,0.00,0.00,0.00,0.00,0.71,19.87,55.20,24.20
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Total,901632.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Simplified example: Generate random losses based on ratings
np.random.seed(42)
n_scenarios = 1000  # Use 20,000 for real cases
n_bonds = len(bonds_filtered)

# Simulate losses (replace with CreditMetrics logic)
loss_factors = np.random.uniform(low=0.3, high=1.2, size=(n_scenarios, n_bonds))
y_scenarios = np.array(bonds_filtered['LTP \n'].values) * loss_factors  # Future values under each scenario
b = bonds_filtered['LTP \n'].values  # Benchmark (no migration)

TypeError: can't multiply sequence by non-int of type 'float'